In [1]:
import sys
sys.path.insert(0, '../../')
import pandas as pd
import random
from tqdm import tqdm_notebook
tqdm_notebook().pandas()
from scripts.utils.data import FB5M_NAME_TABLE
from scripts.utils.data import FB2M_NAME_TABLE
from scripts.utils.data import FB2M_KG_TABLE
from scripts.utils.connect import get_connection 
import scripts.utils.import_notebook
import importlib

connection = get_connection()
cursor = connection.cursor()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [4]:
from scripts.utils.simple_qa import load_simple_qa 

# Load development set because its a magnitude smaller than the training set.
df_dev, = load_simple_qa(dev=True)
df_dev[:5]

,subject,relation,object,question
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...


In [5]:

from functools import partial
edit_distance_link_alias = importlib.import_module(
                "scripts.Simple QA Numbers.HYPOTHESIS - Question Refers to Multiple Subjects").edit_distance_link_alias
normalize = importlib.import_module(
                "scripts.Simple QA Numbers.HYPOTHESIS - Subject Name not in Question").normalize

# Create a column with the subject_name linked per example
df_dev['subject_name'] = df_dev.progress_apply(partial(edit_distance_link_alias, cursor, normalize), axis=1)
print('Dev Linked', sum(df_dev.subject_name.notnull()), 'examples')
display(df_dev[:5])

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))


Dev Linked 10648 examples


,subject,relation,object,question,subject_name
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...,trump ocean club international hotel and tower
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born,sasha vujacic
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in,dead combo
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?,wiebke von carolsfeld
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...,music for stock exchange


In [6]:
import importlib
find_subject_name_span = importlib.import_module(
                "scripts.Simple QA Models.Subject Recognition Data").find_subject_name_span

# Create a column with the subject_name linked per example
df_dev = df_dev.progress_apply(find_subject_name_span, axis=1)
print('Dev:')
display(df_dev[:5])

importing Jupyter notebook from ../../scripts/Simple QA Models/Subject Recognition Data.ipynb


HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))


Dev:


,end_index,object,question,question_tokens,relation,start_index,subject,subject_name,subject_name_tokens
0,10.0,0cqt90,Who was the trump ocean club international hot...,"[who, was, the, trump, ocean, club, internatio...",symbols/namesake/named_after,3.0,0f3xg_,trump ocean club international hotel and tower,"(trump, ocean, club, international, hotel, and..."
1,4.0,0565d,where was sasha vujačić born,"[where, was, sasha, vujacic, born]",people/person/place_of_birth,2.0,07f3jg,sasha vujacic,"(sasha, vujacic)"
2,7.0,07ssc,What is a region that dead combo was released in,"[what, is, a, region, that, dead, combo, was, ...",music/release/region,5.0,031j8nn,dead combo,"(dead, combo)"
3,9.0,0wxsz5y,What is a film directed by wiebke von carolsfeld?,"[what, is, a, film, directed, by, wiebke, von,...",film/director/film,6.0,0c1cyhd,wiebke von carolsfeld,"(wiebke, von, carolsfeld)"
4,7.0,0345h,what country was music for stock exchange rel...,"[what, country, was, music, for, stock, exchan...",music/release/region,3.0,0fvhc0g,music for stock exchange,"(music, for, stock, exchange)"


In [26]:
from tqdm import tqdm_notebook
from collections import defaultdict

def format_question(row):
    ret =  ''
    for i, token in enumerate(row['question_tokens']):
        if i == row['start_index']:
            ret += '<e>'
        elif i > row['start_index'] and i < row['end_index']:
            continue
        else:
            ret += token.lower().strip()
        ret += ' '
    return ret.strip()


question_to_relation = defaultdict(lambda: defaultdict(int))
for index, row in tqdm_notebook(df_dev.iterrows(), total=df_dev.shape[0]):
    if not isinstance(row['subject_name'], str):
        continue
    
    formatted_question = format_question(row)
    question_to_relation[formatted_question][row['relation']] += 1

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))

In [27]:
upperbound = 0
for q, value in question_to_relation.items():
    max_relation_count = max(value.items(), key=lambda item: item[1])[1]
    total_questions = sum(value.values())
    upperbound += max_relation_count
upperbound /= df_dev.shape[0] # - df_dev['subject_name'].isnull().sum()

In [28]:
upperbound

0.909174734900876